<h1>Parsing OMA website</h1>

<p>This notebook is just a prototype for testing pieces of python code aimed to parse a specific website.</p>

In [2]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup
import pandas as pd
import re

In [3]:
url = 'https://www.oma.by/catalog/'
#opens up a connection, grabs the page and downloads it
oma_main_html = requests.get(url)

<p>soup is a BeautifulSoup object which represents the document as a nested data sctructure:</p>

In [4]:
oma_catalog_soup = BSoup(oma_main_html.text, 'html.parser')
oma_catalog_soup.title
oma_catalog_soup.close

<h3>Getting categories </h3>
<p>So we get the soup object successfully. Now will try to find category headers on a catalog page (there are 7 main categories). </p>
<img src="screenshots/category_schema.png" align="left" width="600">

In [5]:
categories = oma_catalog_soup.select('section.bordered-section h2')
print(f'There are {len(categories)} objects that has h2 tags inside the header tag')

print('\nThe whole list of the categories:')
for category in categories:
    print(category)

There are 7 objects that has h2 tags inside the header tag

The whole list of the categories:
<h2>Инструменты, крепёж</h2>
<h2>Отделка</h2>
<h2>Садовый центр / отдых</h2>
<h2>Сантехника</h2>
<h2>Строительство</h2>
<h2>Товары для дома</h2>
<h2>Электротехника</h2>


<h3>Getting all categories</h3>
<p>So we get the categories list correctly and we can use this h2 tags as anchors for looping.</p>
<img src="screenshots/subcategories_schema.png" align="left" width="720">

<p>Getting subcategories lvl1 and subcategores lvl2 is possible by processing each category one by one. It is useful to extract 7 category sections for that. </p>

In [6]:
categories = oma_catalog_soup.findAll('section',\
                                      {'class':'bordered-section js-accordion-group'})

print(f'\nThe type of categories is {type(categories)}.')

print(f'\nThere are {len(categories)} main category objects in {type(categories)}.')

print(f'\nEach category object has {type(categories[0])} type.')

#strip the name of the category
category_name_0 = categories[0].select('section.bordered-section h2')

print('\nFirst category raw:')
print(category_name_0)
print(type(category_name_0))

print('\nFirst category extracted from list:')
print(category_name_0[0])
print(type(category_name_0[0]))

print('\nFirst category extracted from list and converted to string:')
print(str(category_name_0[0]))
print(type(str(category_name_0[0])))
print('')


The type of categories is <class 'bs4.element.ResultSet'>.

There are 7 main category objects in <class 'bs4.element.ResultSet'>.

Each category object has <class 'bs4.element.Tag'> type.

First category raw:
[<h2>Инструменты, крепёж</h2>]
<class 'list'>

First category extracted from list:
<h2>Инструменты, крепёж</h2>
<class 'bs4.element.Tag'>

First category extracted from list and converted to string:
<h2>Инструменты, крепёж</h2>
<class 'str'>



<p>Extract category names:</p>

In [7]:
def remove_tags(text): 
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def extract_category_name(list):
    name_with_tags = str(list[0])
    return remove_tags(name_with_tags)

for category in categories:
    category_name_raw = category.select('section.bordered-section h2')
    print(extract_category_name(category_name_raw))

Инструменты, крепёж
Отделка
Садовый центр / отдых
Сантехника
Строительство
Товары для дома
Электротехника


<h3>Getting subcategories lvl_1</h3>
<p>Getting subcategories lvl_1 fron single category object.</p>

In [8]:
subcats_lvl_1 = categories[0].findAll('div',\
                                      {'class':'catalog-all-item'})

print(f'\nThe type of subcats_lvl_1 is {type(subcats_lvl_1)}.')

print(f'\nThere are {len(subcats_lvl_1)} subcats_lvl_1 objects')

print(f'\nEach subcats_lvl_1 object has {type(subcats_lvl_1[0])} type.')

print('\n')
print(subcats_lvl_1[0])


The type of subcats_lvl_1 is <class 'bs4.element.ResultSet'>.

There are 40 subcats_lvl_1 objects

Each subcats_lvl_1 object has <class 'bs4.element.Tag'> type.


<div class="catalog-all-item col col-lg-3-of-12 col-md-4-of-12 col-sm-6-of-12 col-xs-12-of-12">
<div class="catalog-all-item_img-box">
<img alt="" src="/upload/Sh/imageCache/ad0/5e0/14b9b1eb45799ec953c4d4a00d601b13.jpg"/>
</div>
<div class="accordion-item accordion-item__right-icon js-accordion">
<div class="accordion-item_top">
<div class="accordion-item_icon css-plus-icon js-accordion-bar"></div>
<div class="accordion-item_title">
<a href="/ankery-13127-c">Анкеры</a>
</div>
</div>
<div class="accordion-item_body js-accordion-body js-show-more-box" data-items-visible="11">
<a class="section-submenu-sublink" href="/ankery-ramnye-13251-c">Анкеры рамные</a>
<a class="section-submenu-sublink" href="/ankery-spetsialnye-13252-c">Анкеры специальные</a>
<a class="section-submenu-sublink" href="/bolty-ankernye-13253-c">Болты анкерны

<h3>Getting subcategories lvl_2</h3>
<p>Getting subcategories lvl_2 from single subcats_lvl_1 object.</p>

In [9]:
subcats_lvl_2 = subcats_lvl_1[0].findAll('a',\
                                      {'class':'section-submenu-sublink'})

print(f'\nThe type of subcats_lvl_2 is {type(subcats_lvl_2)}.')

print(f'\nThere are {len(subcats_lvl_2)} subcats_lvl_2 objects')

print(f'\nEach subcats_lvl_2 object has {type(subcats_lvl_2[0])} type.')

print('\n')
print(subcats_lvl_2)


The type of subcats_lvl_2 is <class 'bs4.element.ResultSet'>.

There are 3 subcats_lvl_2 objects

Each subcats_lvl_2 object has <class 'bs4.element.Tag'> type.


[<a class="section-submenu-sublink" href="/ankery-ramnye-13251-c">Анкеры рамные</a>, <a class="section-submenu-sublink" href="/ankery-spetsialnye-13252-c">Анкеры специальные</a>, <a class="section-submenu-sublink" href="/bolty-ankernye-13253-c">Болты анкерные</a>]


<h3>Creating a dataframe containing all links</h3>
<p>Seems like a good idea to have dataframe containing all links to subcategories lvl 2. The dataframe is expected to be lightweight and unseful for future parsing.</p>

In [10]:
main_page_links_df = pd.DataFrame(columns = ['Link number', 'Category',\
                                             'Subcategory lvl 1','Subcategory lvl 2',\
                                             'Link']) 
main_page_links_df

,Link number,Category,Subcategory lvl 1,Subcategory lvl 2,Link
